In [7]:
from datasets import load_dataset

dataset_sample = load_dataset("alexedw/minipile_recreation_tiny", split="train")

Found cached dataset parquet (/Users/alex/.cache/huggingface/datasets/alexedw___parquet/alexedw--minipile_recreation_tiny-ba1d584d949db89d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [11]:
num_shards = 10
import json

dataset_length = len(dataset_sample)

config = {
    "num_shards": num_shards,
    "dataset_length": dataset_length,
}

import os
os.makedirs("dataset_upload.ignore", exist_ok=True)
with open("dataset_upload.ignore/config.json", "w") as f:
    json.dump(config, f)
    
import math
# split dataset into shards
shard_size = math.ceil(dataset_length / num_shards)
for shard_id in range(num_shards):
    start = shard_id * shard_size
    end = min((shard_id + 1) * shard_size, dataset_length)
    
    # Save as a parquet file
    dataset_sample.select(range(start, end)).to_parquet(f"dataset_upload.ignore/shard_{shard_id}.parquet")




Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
from huggingface_hub import HfApi
api = HfApi()

repo_id = "ZelaAI/minipile_test"

api.create_repo(repo_id=repo_id, exist_ok=True, repo_type="dataset")

api.upload_folder(
    repo_id=repo_id,
    folder_path="dataset_upload.ignore",
    repo_type="dataset",
)

'https://huggingface.co/datasets/ZelaAI/minipile_test/tree/main/'

In [39]:
for item in stream_dataset:
    item['text']


Loading shard 1
Loading shard 2
Loading shard 3
Loading shard 4
Loading shard 5
Loading shard 6
Loading shard 7
Loading shard 8
Loading shard 9


In [40]:
from typing import Iterator
from transformers import GPTNeoXTokenizerFast

class TokenizedDatasetWrapper(IterableDataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.tokenizer = GPTNeoXTokenizerFast.from_pretrained("EleutherAI/pythia-410m")

    def __iter__(self) -> Iterator:
        for item in self.dataset:
            yield self.tokenizer.encode(item['text'])

In [41]:
for item in TokenizedDatasetWrapper(HuggingfaceStreamDataset("ZelaAI/minipile_test")):
    item

Loading from dataset with 10000 examples, split into 10 shards of size 1000
Loading shard 0
Loading shard 1
Loading shard 2
Loading shard 3
Loading shard 4
Loading shard 5
Loading shard 6
Loading shard 7
Loading shard 8
Loading shard 9


In [42]:
from torch.utils.data import DataLoader
dataset = TokenizedDatasetWrapper(HuggingfaceStreamDataset("ZelaAI/minipile_test"))
dataloader = DataLoader(dataset, batch_size=2, num_workers=2)

Loading from dataset with 10000 examples, split into 10 shards of size 1000
Loading shard 0


In [43]:
for item in dataloader:
    item

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/alex/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/alex/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'TokenizedDatasetWrapper' on <module '__main__' (built-in)>


KeyboardInterrupt: 

In [ ]:
# Goal is to allow for multiprocessing so we can dynamically load and process data easily as it comes in rather than loading all at once.

In [ ]:
# class SequenceStreamDataset(IterableDataset):
#     def __init__(self, streaming_dataset, skip_first_n=0, insert_dense_tokens=0, tokenizer=None):
#         print(f'Dataset skipping first {skip_first_n} items', flush=True)
#         self.streaming_dataset = streaming_dataset.skip(skip_first_n)
#         self.insert_dense_tokens = insert_dense_tokens
#         self.tokenizer = tokenizer

#     def __iter__(self):
#         for item in self.streaming_dataset:
#             sequences = []
#             for seq in item['packed_sequences']:
#                 context_ids = seq['context_ids']
#                 completion_ids = seq['completion_ids']
                
#                 if self.insert_dense_tokens > 0:
#                     num_to_insert = random.randint(1, self.insert_dense_tokens)

#                     # enforce never more than 1/3 of the sequence is dense tokens
#                     num_to_insert = min(num_to_insert, len(completion_ids)//3)
                    
#                     # shorten completion_ids to make room for dense tokens
#                     completion_ids = completion_ids[:-num_to_insert]

#                     # perform insertions
#                     for i in range(num_to_insert):
#                         index = random.randint(0, len(completion_ids))
#                         completion_ids.insert(index, DENSE_TOKEN_ID)
                
#                 sequences.append(Sequence(
#                     0,
#                     torch.tensor(context_ids, dtype=torch.long),
#                     0,
#                     [torch.tensor(completion_ids, dtype=torch.long)],
#                 ))
#             yield sequences